# WhatsApp sqlite DB tools

### Original Medium post: https://medium.com/@1522933668924/extracting-whatsapp-messages-from-backups-with-code-examples-49186de94ab4
#### License: Free to use, include credit / attribution

In [38]:
import pandas as pd
import sqlite3
import base64
import re

## First, Let's figure out what tables do we have in the DB

In [ ]:
con = sqlite3.connect("ChatStorage.sqlite")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", con)
con.close()
tables

## Since I already looked through those tables I know that the interesting one is ZWAMESSAGE, let's open it and start looking around

In [ ]:
con = sqlite3.connect("ChatStorage.sqlite")
df = pd.read_sql_query("SELECT * from ZWAMESSAGE", con)
con.close()
df.tail(5)

### Let's see what columns are there?

In [ ]:
print('\n'.join([str(i+1)+' '+x for i, x in enumerate(df.columns)]))

In [ ]:
get_df_by_sender = lambda df, sender_id: df[df['ZFROMJID'].str.contains(sender_id, na=False)]

# Example: Get all messages sent by a specific group or person
filtered_df = get_df_by_sender(df, "120363330063276227@g.us")  
print(filtered_df.ZTEXT.tail(10))

## Now that we have a DataFrame let's start by searching based on the phone number

In [49]:
#get df that only contains chats with a specific phone number (or part of it)
get_df_by_number = lambda df, num: df[df.ZTOJID.str.contains(num).fillna(False) | df.ZFROMJID.str.contains(num).fillna(False)]
#note that the fillna is critical to prevent the weird False | None != None | False behavior

In [ ]:
df2 = get_df_by_number(df, '1163')

In [ ]:
#let's check how many messages did we find
len(df2)

In [ ]:
#let's see the text
list(df2.ZTEXT)

In [ ]:
df['decoded_ZPUSHNAME'] = df['ZPUSHNAME'].apply(lambda x: base64.b64decode(x).decode(errors='ignore') if isinstance(x, str) else x)

# Now check the decoded values
print(df[['ZPUSHNAME', 'decoded_ZPUSHNAME']].head(30))

In [ ]:
list(df.decoded_ZPUSHNAME)

In [ ]:
def extract_readable_text(text):
    if not isinstance(text, str):
        return text  # Skip non-string values
    matches = re.findall(r'[a-zA-Z0-9@._-]+', text)  # Extract alphanumeric characters, emails, or phone-like patterns
    return " ".join(matches) if matches else None  # Join found patterns into a single string

df.loc[:, 'cleaned_ZPUSHNAME'] = df2['decoded_ZPUSHNAME'].apply(extract_readable_text)

# Check unique cleaned names
print(df['cleaned_ZPUSHNAME'].unique())

## Ok, it works. Let's add some tools to extract interesting messages and test them

In [59]:
#let's make a few more accessors
get_df_by_contact_name = lambda df, name: df[df.decoded_ZPUSHNAME.str.contains(name).fillna(False)]
get_df_by_chat_session_id = lambda df, sid: df[df.ZCHATSESSION == sid]
grep_for_message_text = lambda df, txt: df[df.ZTEXT.str.contains(txt).fillna(False)]

In [ ]:
get_df_by_contact_name(df2, 'Josh').head(2)

In [ ]:
#Let's find all chat sessions that a specific contact participated in
get_df_by_contact_name(df, 'Emma').ZCHATSESSION.unique()

In [ ]:
#now let's read a specific chat session
get_df_by_chat_session_id(df, 105.0).head(2)

In [ ]:
grep_for_message_text(df, 'URGENT').head(2)

## Ok, next step - let's go figure out how to format dates

In [16]:
from datetime import datetime

In [ ]:
#Let's see if it just works if we try to convert ZWMESSAGEDATE to datetime assuming it is a timestamp
datetime.fromtimestamp(df4.ZMESSAGEDATE.iloc[0])

### Well, that didn't work - how could there be a message from 1985? -- reading around it seems Apple likes to count time on iPhones from 1.1.2001 - Let's see what happens if we add that to our timestamp?

In [ ]:
timestamp_to_apple = lambda x: datetime.fromtimestamp(x) + (datetime(2001,1,1) - datetime.fromtimestamp(0))
timestamp_to_apple(df4.ZMESSAGEDATE.iloc[0])

### It works! Let's add a 'Date' column to our DB to show readable dates and make it the index

In [ ]:
df['Date'] = df.ZMESSAGEDATE.apply(timestamp_to_apple)
df.index = df.Date    # let's make a date index
df = df.sort_index()  # now let's sort by index
df.head()

### hmmm, I got messages from 2012? with no text? 
After looking around it seems to be that these messages indicate the creation dates of chat sessions or groups and are probably received from WhatsApp servers...

## Ok let's try to print everything in a nice readable format

In [ ]:
#get the dataframe of all messages where contact Emma appears and show the top 5
#values in the ZTEXT (message text) column together with index (which we set to date)
get_df_by_chat_session_id(df, 159.0)[['ZTEXT']].head() 

### it is a bit annoying that we can't see the full text, so let's fix it

In [ ]:
pd.set_option('display.max_colwidth', -1)
get_df_by_chat_session_id(df, 159.0)[['ZTEXT']].head() 

# The end - hope it was useful. If it did, please star / fork this repo :)